In [ ]:
!pip install biopython scikit-learn --quiet
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from random import shuffle
def load_sequences(file_paths, window=150, stride=100):
    sequences = []
    labels = []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df
file_paths = {
    "Ebola": "/content/drive/MyDrive/sagar/Ebola_200.fasta",
    "Influenza_A": "/content/drive/MyDrive/sagar/Influenza_A_200.fasta",
    "Influenza_B": "/content/drive/MyDrive/sagar/Influenza_B_200.fasta",
    "SARS_CoV_2": "/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta",
    "Tuberculosis": "/content/drive/MyDrive/sagar/Tuberculosis_200.fasta",
    "Zika": "/content/drive/MyDrive/sagar/Zika_200.fasta",
    "Normal": "/content/drive/MyDrive/sagar/Human_200.fasta"
}
data, labels = load_sequences(file_paths, window=150, stride=100)
print(f"Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]
selector = SelectKBest(mutual_info_classif, k=120)
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
base_learners = [
    ('svm', SVM(kernel='poly', degree=2, C=1.0, gamma=0.01, coef0=0.5, probability=True, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=80, max_depth=10, class_weight='balanced', random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=60, learning_rate=0.05, max_depth=3, random_state=42))
]

meta_learner = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
stack_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1,
    passthrough=True
)
stack_model.fit(X_train_scaled, y_train)
y_pred = stack_model.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"\n Final Stacking Classifier Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.1 MB/s eta 0:00:00
Augmented sequences: 5146

 Final Stacking Classifier Accuracy: 92.23%

 Classification Report:

              precision    recall  f1-score   support

           0       0.93      0.81      0.87       224
           1       0.88      0.83      0.85       130
           2       0.98      0.96      0.97       131
           3       0.93      0.99      0.96       420
           4       0.91      0.98      0.94        86
           5       0.81      0.95      0.88        37
           6       1.00      1.00      1.00         2

    accuracy                           0.92      1030
   macro avg       0.92      0.93      0.92      1030
weighted avg       0.92      0.92      0.92      1030

